# LDA 

## Pre-processing

In [ ]:
import os
import pandas as pd
import re
import numpy as np

iterii=200 # Iterations of LDA model
NPW=12 # Word number under each topic
NTopic=23 # Topic number
Ntmax=50 # Topic number when analyzing perplexity

# Initializing NLTK word capture
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
nltk_stopwords = set(stopwords.words("english"))
# Define a custom list of stop words
custom_stopwords = {
    'the', 'ci','is', 'in', 'and', 'to', 'of', 'for', 'with', 'on', 'a', 'an', 'it', 
    'this', 'that', 'by', 'as', 'at', 'from', 'but', 'or', 'not', 'are', 'was', 
    'be', 'have', 'has', 'had', 'were', 'will', 'would', 'can', 'could', 'should'
}

'''
def english_word_process(text):
    if pd.isna(text): 
        return ""

    # 转换为小写
    text = text.lower()
    
    # 使用 NLTK 进行分词
    tokens = word_tokenize(text)
'''
# Define the processing function
def english_word_process(text):
    if not isinstance(text,str):
        return ""
    # Tokenize by splitting on whitespace and removing non-alphabetic characters
    tokens = [re.sub(r'[^a-zA-Z]', '', word) for word in text.split()]
    
    # Filter tokens
    processed_words = [
        word.lower() for word in tokens 
        if word.lower() not in custom_stopwords and len(word) > 1
    ]
    
    # Return processed text
    return " ".join(processed_words)

# Read in the Excel file and apply the processing function
data = pd.read_excel("./2024.xlsx")  # Replace with your file path
data["content_cutted"] = data.content.apply(english_word_process)

# Display the processed data
#print(data.head())
data["content_cutted"][0]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\92808\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


'article addresses reflection categories development sustainability popular environmental education establishes its conceptual links shows examples incidence popular environmental education cuban environmental policy well contributions political pedagogical nature educational concept sustainable development topic current relevance allows latin american caribbean environmental reality discussed critical perspective faces challenges before construction educational bases sustainable development favor paradigms against hegemonic systems domination prevail modern society'

## LDA analysis

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

def print_top_words(model, feature_names, n_top_words):
    tword = []
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        topic_w = " ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]])
        tword.append(topic_w)
        print(topic_w)
    return tword

n_features = 1000 # Extracing n_features features in topics, default 1000
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                max_features=n_features,
                                stop_words='english',
                                max_df = 0.5,
                                min_df = 10)
tf = tf_vectorizer.fit_transform(data.content_cutted)

n_topics = NTopic
lda = LatentDirichletAllocation(n_components=n_topics, max_iter=iterii,
                                learning_method='batch',
                                learning_offset=50,
#                                 doc_topic_prior=0.1,
#                                 topic_word_prior=0.01,
                               random_state=0)
lda.fit(tf)

,"n_components n_components: int, default=10Number of topics... versionchanged:: 0.19 ``n_topics`` was renamed to ``n_components``",23
,"doc_topic_prior doc_topic_prior: float, default=NonePrior of document topic distribution `theta`. If the value is None,defaults to `1 / n_components`.In [1]_, this is called `alpha`.",None
,"topic_word_prior topic_word_prior: float, default=NonePrior of topic word distribution `beta`. If the value is None, defaultsto `1 / n_components`.In [1]_, this is called `eta`.",None
,"learning_method learning_method: {'batch', 'online'}, default='batch'Method used to update `_component`. Only used in :meth:`fit` method.In general, if the data size is large, the online update will be muchfaster than the batch update.Valid options:- 'batch': Batch variational Bayes method. Use all training data in each EM update. Old `components_` will be overwritten in each iteration.- 'online': Online variational Bayes method. In each EM update, use mini-batch of training data to update the ``components_`` variable incrementally. The learning rate is controlled by the ``learning_decay`` and the ``learning_offset`` parameters... versionchanged:: 0.20 The default learning method is now ``""batch""``.",'batch'
,"learning_decay learning_decay: float, default=0.7It is a parameter that control learning rate in the online learningmethod. The value should be set between (0.5, 1.0] to guaranteeasymptotic convergence. When the value is 0.0 and batch_size is``n_samples``, the update method is same as batch learning. In theliterature, this is called kappa.",0.7
,"learning_offset learning_offset: float, default=10.0A (positive) parameter that downweights early iterations in onlinelearning. It should be greater than 1.0. In the literature, this iscalled tau_0.",50
,"max_iter max_iter: int, default=10The maximum number of passes over the training data (aka epochs).It only impacts the behavior in the :meth:`fit` method, and not the:meth:`partial_fit` method.",10
,"batch_size batch_size: int, default=128Number of documents to use in each EM iteration. Only used in onlinelearning.",128
,"evaluate_every evaluate_every: int, default=-1How often to evaluate perplexity. Only used in `fit` method.set it to 0 or negative number to not evaluate perplexity intraining at all. Evaluating perplexity can help you check convergencein training process, but it will also increase total training time.Evaluating perplexity in every iteration might increase training timeup to two-fold.",-1
,"total_samples total_samples: int, default=1e6Total number of documents. Only used in the :meth:`partial_fit` method.",1000000.0
,"perp_tol perp_tol: float, default=1e-1Perplexity tolerance. Only used when ``evaluate_every`` is greater than 0.",0.1


### Outputing words in each topic

In [3]:
n_top_words = NPW
#tf_feature_names = tf_vectorizer.get_feature_names()
tf_feature_names = tf_vectorizer.get_feature_names_out()
topic_word = print_top_words(lda, tf_feature_names, n_top_words)

Topic #0:
factors results data development research sustainable social health learning students
Topic #1:
social results data research development sustainable factors health students learning
Topic #2:
development sustainable data results social research factors health learning students
Topic #3:
students development sustainable results learning data research social factors health
Topic #4:
students results research data development learning sustainable social health factors
Topic #5:
social development research data health sustainable factors results students learning
Topic #6:
research sustainable data development results students social factors health learning
Topic #7:
development research sustainable results students data social factors learning health
Topic #8:
health development results sustainable data research social factors students learning
Topic #9:
social development research sustainable results students learning data factors health
Topic #10:
students factors research res

### Outputing topics in each article

In [4]:
topics=lda.transform(tf)
topic = []
for t in topics:
    topic.append(list(t).index(np.max(t)))
data['topic']=topic
data.to_excel("data_topic.xlsx",index=False)

topics[0]#0 1 2 
# Probability of each topic
ii=0
for ele in topics[0]:
    print("topic #",ii," probability: ",ele*100,"%")
    ii=ii+1

topic # 0  probability:  0.7246376811882022 %
topic # 1  probability:  0.7246376811594203 %
topic # 2  probability:  84.05797100749156 %
topic # 3  probability:  0.7246376816839066 %
topic # 4  probability:  0.724637681159929 %
topic # 5  probability:  0.7246376811789398 %
topic # 6  probability:  0.7246376817402297 %
topic # 7  probability:  0.7246376819043797 %
topic # 8  probability:  0.7246376812032885 %
topic # 9  probability:  0.7246376816496851 %
topic # 10  probability:  0.7246376811594203 %
topic # 11  probability:  0.7246376811620838 %
topic # 12  probability:  0.7246376812068103 %
topic # 13  probability:  0.7246376811619597 %
topic # 14  probability:  0.7246376815205396 %
topic # 15  probability:  0.7246376822840965 %
topic # 16  probability:  0.7246376812032994 %
topic # 17  probability:  0.7246376821317961 %
topic # 18  probability:  0.7246376827443612 %
topic # 19  probability:  0.7246376811594203 %
topic # 20  probability:  0.7246376812198598 %
topic # 21  probability: 

## Visualization

In [5]:
import pyLDAvis
#import pyLDAvis.sklearn
import pyLDAvis.lda_model

pyLDAvis.enable_notebook()
pic = pyLDAvis.lda_model.prepare(lda, tf, tf_vectorizer)
pyLDAvis.save_html(pic, 'lda_pass'+str(n_topics)+'.html')
#pyLDAvis.show(pic)

topic_strength = data['topic'].value_counts(normalize=True).to_dict()
print("Topic Strength:")
for topic_id, strength in sorted(topic_strength.items()):
    print(f"Topic {topic_id} = ({strength:.8f})")
ii=0

Topic Strength:
Topic 0 = (0.11481236)
Topic 1 = (0.01554123)
Topic 2 = (0.09800614)
Topic 3 = (0.04728631)
Topic 4 = (0.06981507)
Topic 5 = (0.05047889)
Topic 6 = (0.01463767)
Topic 7 = (0.04439492)
Topic 8 = (0.08336847)
Topic 9 = (0.02301066)
Topic 10 = (0.01373411)
Topic 11 = (0.04812963)
Topic 12 = (0.06608036)
Topic 13 = (0.06848985)
Topic 14 = (0.00909584)
Topic 15 = (0.02258900)
Topic 16 = (0.04379254)
Topic 17 = (0.04523824)
Topic 18 = (0.03843142)
Topic 19 = (0.03017890)
Topic 20 = (0.01981808)
Topic 21 = (0.00758990)
Topic 22 = (0.02548039)
